### **MNIST**

### **LIBRERIE & IMPORT**

In [3]:
# Librerie
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

################ SOLO SU VS CODE ##################
#Funzione di plotting
# import os, sys
#sys.path.append(os.path.abspath(".."))
# from FUNCTIONS.utils import <nome-funzione>
###################################################

# Funzione che serve ad effettuare la divisione tra train set e test set
from sklearn.model_selection import train_test_split
# Metriche per definire la qualità del modello generato
from sklearn.metrics import accuracy_score #, mean_squared_error, classification_report
# Caricamento Dati
from sklearn.datasets import fetch_openml
from sklearn.datasets import load_digits
# Per Scaling dei dati
from sklearn.preprocessing import StandardScaler
# Decision Tree
from sklearn.tree import DecisionTreeClassifier, plot_tree
# Logistic Regression
from sklearn.linear_model import LogisticRegression
# SVM
from sklearn.svm import SVC
# NEURAL NETWORK
from sklearn.neural_network import MLPClassifier
# KNN
from sklearn.neighbors import KNeighborsClassifier
# KMEANS
from sklearn.cluster import KMeans

### **DATASET MNIST**

In [2]:

RANDOM_SEED = 42

# Versione completa, 28x28
mnist = fetch_openml('mnist_784')
# Versione ridotta, 8x8
# mnist = load_digits()

X, y = mnist.data, mnist.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, reandom_state = RANDOM_SEED)


SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (ipython-input-2771139647.py, line 10)